## 이 문서를 수정할 당신에게...
#### 현재 상황은 아래와 같습니다.
1. 현재 Assistant와 Agent에 대해 evaluation set과 few shot examples를 위한 우리 챗봇내에서 일어날만한 대화 데이터셋을 제작해야합니다.

#### 당신의 목표는 아래와 같습니다.
1. 최대한 우리 챗봇에서 일어날만한 대화 데이터셋 100개 제작을 목표로 합니다.

#### 현재 제가 생각한 방식입니다.
1. GPT4 API를 사용한 데이터셋 제작
- GPT 3.5는 한국어에 대한 성능과 대화셋 제작에 대한 성능이 부족하기 때문에 GPT4 API를 활용합니다.
- 아래 논문을 확인하여 프롬프트를 작성하고, 데이터셋을 수집합니다.
[Building a Role Specified Open-Domain Dialogue System Leveraging Large-Scale Language Models 논문](https://arxiv.org/pdf/2205.00176.pdf)
- 선택사항이지만 확장성을 위해 Azure OpenAI API보다 Langchain을 활용해 개발하는 것을 추천합니다.

#### 예상되는 문제
1. 데이터 수집에서 바닐라 GPT-4와 실제로 우리가 사용하는 챗봇은 다릅니다. 또한 실제 유저가 하는 말과 GPT-4가 하는 말은 다릅니다. 이로인해 실제 배포 환경과 다른 대화 셋입니다. 이 때문에 일단은 적은 수의 대화셋만 수집합니다. 
2. 우리의 챗봇은 실제 DB에 있는 데이터를 사용하는 등의 외부의 데이터를 사용하여 말을 생성합니다. 이 또한 실제 환경과 다른 데이터셋이 생성될 것입니다.

#### Future works
1. 실제 데이터와의 괴리를 줄이기 위해 우리의 챗봇과 사람이 대화할 수 있는 UI를 제작합니다.
2. 유저 프롬프트 등을 작성하여 실제 유저와 최대한 유사한 유저 언어모델을 돌릴 수 있도록 합니다. 즉 데이터셋 생성에서 유저 언어모델과 챗봇 언어모델 2가지의 모델이 돌아가면서 대화할 수 있도록 합니다.
3. 유저에 다양한 페르소나를 부여하여 좀 더 다양성있는 데이터를 수집하도록 합니다.
4. 언어모델의 문제 중에 hallucination(거짓말)과 성별,종교에 편향되거나 폭력적인 말을 하는 문제가 있습니다. 그리고 이를 유도하는 유저의 말이 있습니다. 이런 것들만을 위한 대화 데이터셋도 구성될 필요가 있습니다.(최근 네이버에서 이런 문제를 유발하는 한국어 데이터셋을 공개하기도 하고, 이에 관련한 영어 데이터셋도 많이 존재합니다.)

#### 참고 사항
아래는 위의 작업을 수행하기 위한 좋은 자료들입니다. 위의 문제해결 전에 아래 자료들을 먼저 학습하는 것을 매우 강력하게 추천합니다.
- [프롬프트 엔지니어링 강의 2시간](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/)
- [프롬프트 엔지니어링 가이드 documentation](https://www.promptingguide.ai/techniques)

아래 논문은 직접적이진 않지만 비싼 사람 데이터대신 언어모델을 사용하여 데이터를 수집하는 방식에 근간이 된 논문입니다.
- [Self-Instruct: Aligning Language Models with Self-Generated Instructions](https://arxiv.org/pdf/2212.10560.pdf)

----------------------------------------------------------------

### API 키 불러오기

In [ ]:
import os
import configparser
import time
import copy

In [ ]:
config = configparser.ConfigParser()
config.read('./secrets.ini')

['./secrets.ini']

In [ ]:
openai_api_key = config['OPENAI']['OPENAI_API_KEY']
os.environ.update({'OPENAI_API_KEY': openai_api_key})